In [1]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import Normalizer, MinMaxScaler

In [2]:
tracks = pd.read_csv('D:\\fma_metadata\\tracks.csv')
genres = pd.read_csv('D:\\fma_metadata\\genres.csv')
features = pd.read_csv('D:\\fma_metadata\\features.csv')
echonest = pd.read_csv('D:\\fma_metadata\\echonest.csv')
raw_albums = pd.read_csv('D:\\fma_metadata\\raw_albums.csv')
raw_artists = pd.read_csv('D:\\fma_metadata\\raw_artists.csv')
raw_genres = pd.read_csv('D:\\fma_metadata\\raw_genres.csv')
raw_tracks = pd.read_csv('D:\\fma_metadata\\raw_tracks.csv')
raw_echonest = pd.read_csv('D:\\fma_metadata\\raw_echonest.csv')

C:\Users\Arhum Khan\AppData\Local\Temp\ipykernel_6044\1207612644.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv('D:\\fma_metadata\\tracks.csv')
C:\Users\Arhum Khan\AppData\Local\Temp\ipykernel_6044\1207612644.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185

In [3]:
# finding a specifc artist in the raw_artist by using the name and if not found display a message
def find_artist(artist_name):
    # convert the artist_name column to lower case
    artist_name = artist_name.lower()
    raw_artists['artist_name'] = raw_artists['artist_name'].str.lower()
    
    # check if the artist_name is in the raw_artists dataframe
    if artist_name.lower() in raw_artists['artist_name'].values:
        print('Artist found')
    else:
        print('Artist not found')
        
        
artist_name = 'lucky dragons'
find_artist(artist_name)

Artist found


In [8]:
# Path to the directory containing audio files
directory_path = "D:\\fma_small1"

# Initialize lists to store features and corresponding labels
mfcc_features_list = []
labels = []

# Iterate through each folder (genre) in the directory
for folder_name in os.listdir(directory_path):
    genre_folder_path = os.path.join(directory_path, folder_name)
    
    # Iterate through each audio file in the genre folder
    for audio_file_name in os.listdir(genre_folder_path):
        audio_file_path = os.path.join(genre_folder_path, audio_file_name)
        
        # Load audio file and extract MFCC features
        audio_signal, sampling_rate = librosa.load(audio_file_path, sr=None)
        mfcc_features = librosa.feature.mfcc(y=audio_signal, sr=sampling_rate, n_mfcc=13)
        spectral_centroid = librosa.feature.spectral_centroid(y=audio_signal, sr=sampling_rate)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_signal)

        # Append MFCC features and corresponding label to lists
        mfcc_features_list.append(mfcc_features.T)  # Transpose to have features along rows
        labels.append(folder_name)

# Concatenate MFCC features to create a single array
mfcc_features_array = np.concatenate(mfcc_features_list)
spectral_centroid_array = np.concatenate(spectral_centroid)
zero_crossing_rate_array = np.concatenate(zero_crossing_rate)

# Display the shape of the MFCC features array and labels
print(mfcc_features_array.shape)

(1459283, 13)


In [9]:
# RobustScaler for standardization
robust_scaler = MinMaxScaler()
robust_standardized_mfcc_features = robust_scaler.fit_transform(mfcc_features)

# # Normalizer the stndardized features
normalizer = Normalizer("l2")
normalized_spectral_centroid = normalizer.fit_transform(spectral_centroid)
normalized_zero_crossing_rate = normalizer.fit_transform(zero_crossing_rate)

# Display the first 5 rows of the normalized MFCC features
print("Standardized MFCC Features: ",robust_standardized_mfcc_features[:5])
print("________________________")
print("Normalized Spectral Centrioid: ",normalized_spectral_centroid[:5])
print("________________________")
print("Normalized Zero Crossing Rate: ",normalized_zero_crossing_rate[:5])

Standardized MFCC Features:  [[0.         0.         0.         ... 0.         0.         0.        ]
 [1.         0.99975586 0.99999994 ... 1.         1.         1.        ]
 [1.         0.9972293  0.9869605  ... 0.60836256 0.55860186 0.5328129 ]
 [1.         0.9971193  0.9599682  ... 0.6172108  0.55409044 0.553376  ]
 [1.         1.         0.9354352  ... 0.4229007  0.4084065  0.40759063]]
________________________
Normalized Spectral Centrioid:  [[0.03043324 0.04335429 0.03850497 ... 0.01715039 0.01318183 0.01042397]]
________________________
Normalized Zero Crossing Rate:  [[0.00359083 0.01968306 0.04149401 ... 0.01436331 0.01024051 0.00691567]]


In [ ]:
# applying pca on the features
from sklearn.decomposition import PCA

# Initialize PCA with 2 components
pca = PCA(n_components=2)

# Fit and transform the standardized MFCC features
pca_mfcc_features = pca.fit_transform(robust_standardized_mfcc_features)

# Display the shape of the PCA features
print(pca_mfcc_features)

[[ 29.064919     0.4523835 ]
 [-21.38128     -2.5689156 ]
 [ -5.5007195    1.8427001 ]
 [ -6.301896     3.268662  ]
 [  3.4497154   -2.9754572 ]
 [ -4.182367     2.6037693 ]
 [  3.8776062   -1.5454011 ]
 [ -3.3769104    0.82872695]
 [  3.2919517   -1.5375692 ]
 [  0.32416484   0.9588867 ]
 [  1.7882175   -1.3476554 ]
 [ -1.1445957    0.1909703 ]
 [  0.09119833  -0.17110173]]


In [ ]:
# save the normalized features to a npy
np.save('normalized_mfcc_features.npy', robust_standardized_mfcc_features)
np.save('normalized_spectral_centroid.npy', normalized_spectral_centroid)
np.save('normalized_zero_crossing_rate.npy', normalized_zero_crossing_rate)